# 스크립트를 파이프라인 작업으로 실행

파이프라인을 사용하면 여러 단계를 하나의 워크플로로 그룹화할 수 있습니다. 구성 요소를 사용하여 파이프라인을 빌드할 수 있습니다. 각 구성 요소는 실행할 Python 스크립트를 반영합니다. 구성 요소는 스크립트 및 실행 방법을 지정하는 YAML 파일에 정의됩니다. 

## 시작하기 전에

이 Notebook에서 코드를 실행하려면  **최신 버전의 azureml-ai-ml** 패키지가 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

> **고**:
> **azure-ai-ml** 패키지가 설치되지 않은 경우 를 실행 `pip install azure-ai-ml` 하여 설치합니다.

In [ ]:
## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

작업 영역에 연결하려면 식별자 매개 변수(구독 ID, 리소스 그룹 이름 및 작업 영역 이름)가 필요합니다. 리소스 그룹 이름 및 작업 영역 이름이 이미 채워져 있습니다. 명령을 완료하려면 구독 ID만 필요합니다.

필요한 매개 변수를 찾으려면 Studio의 오른쪽 위에 있는 구독 및 작업 영역 이름을 클릭합니다. 오른쪽에 창이 열립니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 구독 ID를 복사하고 **YOUR-SUBSCRIPTION-ID** 를 복사한 값으로 바꿉니다. </p>

## 스크립트 만들기

다음 두 단계로 파이프라인을 빌드합니다.

1. **데이터 준비**: 누락된 데이터를 수정하고 데이터를 정규화합니다.
1. **모델 학습**: 의사 결정 트리 분류 모델을 학습합니다.

다음 셀을 실행하여 **src** 폴더와 두 스크립트를 만듭니다.

In [ ]:
## 구성 요소 정의

구성 요소를 정의하려면 다음을 지정해야 합니다.

- **메타데이터**: *이름*, *표시 이름*, *버전*, *설명*, *형식* 등 메타데이터는 구성 요소를 설명하고 관리하는 데 도움이 됩니다.
- **인터페이스**: *입력 및* *출력.* 예를 들어 모델 학습 구성 요소는 학습 데이터와 정규화 속도를 입력으로 사용하고 학습된 모델 파일을 출력으로 생성합니다. 
- **명령, 코드 & 환경**: 구성 요소를 실행하는 *명령*, *코드* 및 *환경* 입니다. 명령은 구성 요소를 실행하는 셸 명령입니다. 코드는 일반적으로 소스 코드 디렉터리를 참조합니다. 환경은 AzureML 환경(큐레이팅 또는 사용자 지정 생성), Docker 이미지 또는 conda 환경일 수 있습니다.

다음 셀을 실행하여 파이프라인 단계로 실행하려는 각 구성 요소에 대한 YAML을 만듭니다.

In [ ]:
## 구성 요소 로드

이제 각 구성 요소를 정의했으므로 YAML 파일을 참조하여 구성 요소를 로드할 수 있습니다. 

## 파이프라인 빌드

구성 요소를 만들고 로드한 후 파이프라인을 빌드할 수 있습니다. 두 구성 요소를 파이프라인으로 구성합니다. `prep_data` 먼저 구성 요소를 실행하려고 합니다. 첫 번째 구성 요소의 출력은 모델을 학습시키는 두 번째 구성 요소 `train_decision_tree`의 입력이어야 합니다.

함수는 `diabetes_classification` 전체 파이프라인을 나타냅니다. 함수에는 하나의 입력 변수인 가 `pipeline_job_input`예상됩니다. 설정 중에 데이터 자산이 만들어졌습니다. 등록된 데이터 자산을 파이프라인 입력으로 사용합니다. 

In [ ]:
개체를 인쇄하여 파이프라인 작업의 구성을 검색할 수 있습니다.`pipeline_job`

In [ ]:
매개 변수를 참조하고 새 값을 지정하여 파이프라인 작업 구성의 매개 변수를 변경할 수 있습니다.

In [ ]:
## 파이프라인 작업 제출

마지막으로 파이프라인을 빌드하고 필요에 따라 실행되도록 파이프라인 작업을 구성한 경우 파이프라인 작업을 제출할 수 있습니다.

## Define the components

To define the component you need to specify:

- **Metadata**: *name*, *display name*, *version*, *description*, *type* etc. The metadata helps to describe and manage the component.
- **Interface**: *inputs* and *outputs*. For example, a model training component will take training data and the regularization rate as input, and generate a trained model file as output. 
- **Command, code & environment**: the *command*, *code* and *environment* to run the component. Command is the shell command to execute the component. Code usually refers to a source code directory. Environment could be an AzureML environment (curated or custom created), docker image or conda environment.

Run the following cells to create a YAML for each component you want to run as a pipeline step.

In [ ]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

In [ ]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train a decision tree classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_file
  reg_rate:
    type: number
    default: 0.01
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --reg_rate ${{inputs.reg_rate}} 
  --model_output ${{outputs.model_output}} 

## Load the components

Now that you have defined each component, you can load the components by referring to the YAML files. 

In [ ]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_decision_tree = load_component(source=parent_dir + "./train-model.yml")

## Build the pipeline

After creating and loading the components, you can build the pipeline. You'll compose the two components into a pipeline. First, you'll want the `prep_data` component to run. The output of the first component should be the input of the second component `train_decision_tree`, which will train the model.

The `diabetes_classification` function represents the complete pipeline. The function expects one input variable: `pipeline_job_input`. A data asset was created during setup. You'll use the registered data asset as the pipeline input. 

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_decision_tree(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data:1"))

You can retrieve the configuration of the pipeline job by printing the `pipeline_job` object:

In [ ]:
print(pipeline_job)

You can change any parameter of the pipeline job configuration by referring to the parameter and specifying the new value:

In [ ]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

## Submit the pipeline job

Finally, when you've built the pipeline and configured the pipeline job to run as required, you can submit the pipeline job:

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes"
)
pipeline_job